In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Incidents

loop

In [2]:
def resave_incs(path, dest_path):
    pth = Path(path)
    for child in pth.iterdir():
        # incidents data reading
        incdnt_file_name = child.name
        inc_header = ['IncidentID', 'CC_Code', 'Incident_No', 'Timestamp', 'Description', 'Location', 'Area', 'Zoom_Map', 'TBxy', 'Latitude', 'Longitude', 'District', 'CountryFIPS_ID', 'CityFIPS_ID', 'Freeway', 'Freeway_direction', 'State_postmile', 'Absolute_postmile', 'Severity', 'Duration']
        data_inc_d07 = pd.read_csv(path+'/'+incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
        data_inc_d07 = (data_inc_d07.dropna(subset=['District']))[data_inc_d07.columns[:-4]]
        data_inc_d07 = data_inc_d07.astype(dtype={'District':int})
        # result: incidents in district under analysis
        data_inc_d07 = data_inc_d07[data_inc_d07['District']==7].reset_index(drop=True)
        data_inc_d07.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)

In [3]:
resave_incs('../data/PeMS/Incidents/work_folder/Months/inc/inc/raw/', '../data/PeMS/Incidents/work_folder/Months/inc/inc/light/')

In [4]:
raw_path = '../data/PeMS/Incidents/test_env/inc/inc/'
dest_path = '../data/PeMS/Incidents/test_env/inc/light/'
pth = Path(raw_path)
for child in pth.iterdir():
    # incidents data reading
    incdnt_file_name = child.name
    inc_header = ['IncidentID', 'CC_Code', 'Incident_No', 'Timestamp', 'Description', 'Location', 'Area', 'Zoom_Map', 'TBxy', 'Latitude', 'Longitude', 'District', 'CountryFIPS_ID', 'CityFIPS_ID', 'Freeway', 'Freeway_direction', 'State_postmile', 'Absolute_postmile', 'Severity', 'Duration']
    data_inc_d07 = pd.read_csv(raw_path+'/'+incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
    data_inc_d07 = (data_inc_d07.dropna(subset=['District']))[data_inc_d07.columns[:-4]]
    data_inc_d07 = data_inc_d07.astype(dtype={'District':int})
    # result: incidents in district under analysis
    data_inc_d07 = data_inc_d07[data_inc_d07['District']==7].reset_index(drop=True)
    data_inc_d07.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)

single

In [2]:
# incidents data reading
incdnt_file_name = '../data/PeMS/Incidents/raw/all_text_chp_incident_day_2017_03_06.txt.gz'
inc_header = ['IncidentID', 'CC_Code', 'Incident_No', 'Timestamp', 'Description', 'Location', 'Area', 'Zoom_Map', 'TBxy', 'Latitude', 'Longitude', 'District', 'CountryFIPS_ID', 'CityFIPS_ID', 'Freeway', 'Freeway_direction', 'State_postmile', 'Absolute_postmile', 'Severity', 'Duration', 'Incident_ID', 'Detail_ID', 'Timestamp', 'description']
data_inc_d07 = pd.read_csv(incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
data_inc_d07 = (data_inc_d07.dropna(subset=['District']))[data_inc_d07.columns[:-4]]
data_inc_d07 = data_inc_d07.astype(dtype={'District':int})
# result: incidents in district under analysis
data_inc_d07 = data_inc_d07[data_inc_d07['District']==7].reset_index(drop=True)

In [3]:
data_inc_d07.to_csv('../data/PeMS/Incidents/light/all_text_chp_incident_day_2017_03_06.csv', index=False)

# Incidents details

In [5]:
def pre_filter_desc(path, dest_path):
    pth = Path(path)
    det_header = ['ID', 'DetID', 'Timestamp', 'Desc']
    for child in pth.iterdir():
        incdnt_det_file_name = path + '/' + child.name
        det_month = pd.read_csv(incdnt_det_file_name, sep=',', names=det_header, parse_dates=[2])
        det_month.dropna(axis=0, inplace=True)
        det_month = det_month.astype({'ID': np.int, 'DetID': np.int})
        det_month.to_csv(dest_path + '/' + child.name[:-4] + '.csv', index=False)

In [6]:
pre_filter_desc('../data/PeMS/Incidents/work_folder/det/raw/', '../data/PeMS/Incidents/work_folder/det/light')

# Filtering of line blocking incidents

In [13]:
def filter_blk_desc(path):
    dest_path = path + '/inc/blkg'
    bound_re = re.compile(r"(IF [E,N,S,W]B)")
    m_path = path + '/det'
    pth = Path(m_path)
    det_header = ['ID', 'DetID', 'Timestamp', 'Desc']
    for child in pth.iterdir():
        incdnt_det_file_name = m_path + '/' + child.name
        det_month = pd.read_csv(incdnt_det_file_name, sep=',', names=det_header, parse_dates=['Timestamp'])
        det_month.dropna(axis=0, inplace=True)
        det_month = det_month.astype({'ID': np.int, 'DetID': np.int})
        blkg_desc = det_month[det_month.apply(lambda row: ('NOT BLK' not in row.Desc) & ((' BLKG' in row.Desc) | (' BLKD' in row.Desc) | (' BLKING' in row.Desc) | (' BLKG' in row.Desc)) & ((' ENTIR' in row.Desc) | (' LN' in row.Desc)) & (('IF LN' not in row.Desc)) & (not bound_re.search(row.Desc)), axis=1)].copy()
        blkg_desc.drop_duplicates(subset=['ID'], keep='first', inplace=True)
        blkg_desc.to_csv(dest_path + '/' + child.name[:-4] + '.csv', index=False)

In [14]:
dir_list = [
            '../data/PeMS/Incidents/work_folder/Months/inc/',
            ]
for a_dir in dir_list:
    filter_blk_desc(a_dir)

In [ ]:
dir_list = [
            '../data/PeMS/Incidents/work_folder/Months/Jan/',
            '../data/PeMS/Incidents/work_folder/Months/Feb/',
            '../data/PeMS/Incidents/work_folder/Months/Mar/',
            '../data/PeMS/Incidents/work_folder/Months/Apr/',
           '../data/PeMS/Incidents/work_folder/Months/May/',
           '../data/PeMS/Incidents/work_folder/Months/Jun/',
           '../data/PeMS/Incidents/work_folder/Months/Jul/',
           '../data/PeMS/Incidents/work_folder/Months/Aug/',
           '../data/PeMS/Incidents/work_folder/Months/Sep/',
           '../data/PeMS/Incidents/work_folder/Months/Oct/',
           '../data/PeMS/Incidents/work_folder/Months/Nov/',
           '../data/PeMS/Incidents/work_folder/Months/Dec/']
for a_dir in dir_list:
    filter_blk_desc(a_dir)

# Series

loop

In [3]:
raw_path = '../data/PeMS/Series/raw'
raw_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/2017/Dec/'
dest_path = '../data/PeMS/Series/light'
dest_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/light/Dec'
pth = Path(raw_path)
st_blacklist = np.array([])
header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
          'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
          'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
          'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
          'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
          'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
          'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
          'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
          'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
          'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']
for child in pth.iterdir():
    # incidents data reading
    srs_file_name = child.name
#     if srs_file_name[0] !='p':
#         continue
    print(srs_file_name)

    data_srs_smoothed = pd.read_csv(raw_path+'/'+srs_file_name, sep=',', names=header_srs, parse_dates=[0])

    for col in data_srs_smoothed.columns[-40:]:
        del data_srs_smoothed[col]
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)
    data_srs_smoothed.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)
    #smoothing
#     data_srs_smoothed = data_srs_smoothed[data_srs_smoothed.columns[:16]]
#     del data_srs_no_null

    gb = data_srs_smoothed.groupby(by=['Station'])
    j=0
    for i, item in gb:
    #     print(item['Avg Speed'].rolling(5).mean())
    #     station = item.Station[0]
        data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
    #     j += 1
    #     if j==2:
    #         break
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)

#     del srs_for_smoothing_gb

    gb = data_srs_smoothed.groupby(by=['Station'])
    non_sensitive_sts = gb.filter(lambda x: (x['Avg Speed'].std()<1))['Station'].unique()
    data_srs_smoothed = data_srs_smoothed[~data_srs_smoothed.Station.isin(non_sensitive_sts)]
    data_srs_smoothed.to_csv(dest_path+'/smoothed/'+child.name[:-4]+'.csv', index=False)

    data_srs_smoothed = None

    st_blacklist = np.append(st_blacklist, non_sensitive_sts[~np.isin(non_sensitive_sts, st_blacklist)])
    gb, non_sensitive_sts = None, None

d07_text_station_5min_2017_12_22.txt.gz


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "pandas/_libs/tslib.pyx", line 2270, in pandas._libs.tslib.array_to_datetime
  File "pandas/_libs/src/datetime.pxd", line 119, in datetime._string_to_dts
ValueError: Error parsing datetime string "12/22/2017 04:00:00" at position 2

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 3022, in converter
    infer_datetime_format=infer_datetime_format
  File "/opt/anaconda3/lib/python3.6/site-packages/pandas/core/tools/datetimes.py", line 380, in to_datetime
    result = _convert_listlike(arg, box, format)
  File "/opt/anaconda3/lib/python3.6/site-packages/pandas/core/tools/datetimes.py", line 294, in _convert_listlike
    require_iso8601=require_iso8601
  File "pandas/_libs/tslib.pyx", line 2156, in pandas._libs.tslib.array_to_datetime
  File "pandas/_libs/tslib.pyx", line 2292, in pandas._libs.tslib.array_to_datetim


KeyboardInterrupt



In [16]:
def convert_srs_to_csv(path):
    raw_path = path+'/raw/'
    dest_path = path + '/light/'
    pth = Path(raw_path)
    st_blacklist = np.array([])
    header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
              'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
              'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
              'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
              'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
              'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
              'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
              'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
              'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
              'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']
    for child in pth.iterdir():
        # incidents data reading
        srs_file_name = child.name
    #     if srs_file_name[0] !='p':
    #         continue
        print(srs_file_name)

        data_srs_smoothed = pd.read_csv(raw_path+'/'+srs_file_name, sep=',', names=header_srs, parse_dates=[0])

        for col in data_srs_smoothed.columns[-40:]:
            del data_srs_smoothed[col]
        data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)
        data_srs_smoothed.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)

In [18]:
convert_srs_to_csv('../data/PeMS/Incidents/work_folder/Months/inc/series')

d07_text_station_5min_2017_04_28.txt.gz


KeyboardInterrupt: 

In [ ]:
np.savetxt(dest_path+'/stations_blacklist_dec.csv', st_blacklist)

In [5]:
test = pd.read_csv('../data/PeMS/Incidents/test_env/series/d07_text_station_5min_2017_03_01.tx.csv', parse_dates=[0])

In [3]:
def smooth_srs(src_pth):
    pth = Path(src_pth + '/light')
    st_blacklist = np.array([])
    for child in pth.iterdir():
        # incidents data reading
        srs_file_name = child.name
        print(srs_file_name)

        data_srs_smoothed = pd.read_csv(src_pth + '/light/' + srs_file_name, sep=',', parse_dates=[0])

        data_srs_smoothed.dropna(subset=['Avg Speed', 'Avg Occupancy', 'Total Flow'], inplace=True)

        gb = data_srs_smoothed.groupby(by=['Station'])
        j=0
        for i, item in gb:

            data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
            data_srs_smoothed.loc[item.index, ('Avg Occupancy')] = item['Avg Occupancy'].rolling(5).mean()
            data_srs_smoothed.loc[item.index, ('Total Flow')] = item['Total Flow'].rolling(5).mean()

        data_srs_smoothed.dropna(subset=['Avg Speed', 'Avg Occupancy', 'Total Flow'], inplace=True)


        gb = data_srs_smoothed.groupby(by=['Station'])
        non_sensitive_sts = gb.filter(lambda x: (x['Avg Speed'].std()<1))['Station'].unique()
#         data_srs_smoothed = data_srs_smoothed[~data_srs_smoothed.Station.isin(non_sensitive_sts)]
        data_srs_smoothed.to_csv(src_pth + '/smoothed/' + child.name[:-4]+'.csv', index=False)

        st_blacklist = np.append(st_blacklist, non_sensitive_sts[~np.isin(non_sensitive_sts, st_blacklist)])
    np.savetxt(src_pth + '/st_blacklist/stations_blacklist.csv', st_blacklist)

In [7]:
smooth_srs('../data/PeMS/Incidents/work_folder/Months/Jan/series')

d07_text_station_5min_2017_01_08.tx.csv
d07_text_station_5min_2017_01_20.tx.csv
d07_text_station_5min_2017_01_04.tx.csv
d07_text_station_5min_2017_01_02.tx.csv
d07_text_station_5min_2017_01_05.tx.csv
d07_text_station_5min_2017_01_06.tx.csv
d07_text_station_5min_2017_01_21.tx.csv
d07_text_station_5min_2017_01_11.tx.csv
d07_text_station_5min_2017_01_18.tx.csv
d07_text_station_5min_2017_01_09.tx.csv
d07_text_station_5min_2017_01_29.tx.csv
d07_text_station_5min_2017_01_13.tx.csv
d07_text_station_5min_2017_01_23.tx.csv
d07_text_station_5min_2017_01_19.tx.csv
d07_text_station_5min_2017_01_24.tx.csv
d07_text_station_5min_2017_01_03.tx.csv
d07_text_station_5min_2017_01_07.tx.csv
d07_text_station_5min_2017_01_25.tx.csv
d07_text_station_5min_2017_01_26.tx.csv
d07_text_station_5min_2017_01_14.tx.csv
d07_text_station_5min_2017_01_12.tx.csv
d07_text_station_5min_2017_01_01.tx.csv
d07_text_station_5min_2017_01_15.tx.csv
d07_text_station_5min_2017_01_16.tx.csv
d07_text_station_5min_2017_01_30.tx.csv


In [4]:
dir_list = [
            #'../data/PeMS/Incidents/work_folder/Months/Feb/series',
#             '../data/PeMS/Incidents/work_folder/Months/Mar/series',
#             '../data/PeMS/Incidents/work_folder/Months/Apr/series',
#            '../data/PeMS/Incidents/work_folder/Months/May/series',
           '../data/PeMS/Incidents/work_folder/Months/Jun/series',
           '../data/PeMS/Incidents/work_folder/Months/Jul/series',
           '../data/PeMS/Incidents/work_folder/Months/Aug/series',
           '../data/PeMS/Incidents/work_folder/Months/Sep/series',
           '../data/PeMS/Incidents/work_folder/Months/Oct/series',
           '../data/PeMS/Incidents/work_folder/Months/Nov/series',
           '../data/PeMS/Incidents/work_folder/Months/Dec/series']
for a_dir in dir_list:
    smooth_srs(a_dir)

d07_text_station_5min_2017_06_05.tx.csv
d07_text_station_5min_2017_06_02.tx.csv
d07_text_station_5min_2017_06_29.tx.csv
d07_text_station_5min_2017_06_10.tx.csv
d07_text_station_5min_2017_06_17.tx.csv
d07_text_station_5min_2017_06_20.tx.csv
d07_text_station_5min_2017_06_24.tx.csv
d07_text_station_5min_2017_06_09.tx.csv
d07_text_station_5min_2017_06_21.tx.csv
d07_text_station_5min_2017_06_22.tx.csv
d07_text_station_5min_2017_06_28.tx.csv
d07_text_station_5min_2017_06_23.tx.csv
d07_text_station_5min_2017_06_06.tx.csv
d07_text_station_5min_2017_06_19.tx.csv
d07_text_station_5min_2017_06_08.tx.csv
d07_text_station_5min_2017_06_11.tx.csv
d07_text_station_5min_2017_06_18.tx.csv
d07_text_station_5min_2017_06_15.tx.csv
d07_text_station_5min_2017_06_13.tx.csv
d07_text_station_5min_2017_06_30.tx.csv
d07_text_station_5min_2017_06_03.tx.csv
d07_text_station_5min_2017_06_12.tx.csv
d07_text_station_5min_2017_06_26.tx.csv
d07_text_station_5min_2017_06_27.tx.csv
d07_text_station_5min_2017_06_25.tx.csv


d07_text_station_5min_2017_12_08.tx.csv
d07_text_station_5min_2017_12_21.tx.csv
d07_text_station_5min_2017_12_23.tx.csv
d07_text_station_5min_2017_12_03.tx.csv
d07_text_station_5min_2017_12_28.tx.csv
d07_text_station_5min_2017_12_05.tx.csv
d07_text_station_5min_2017_12_17.tx.csv
d07_text_station_5min_2017_12_18.tx.csv
d07_text_station_5min_2017_12_11.tx.csv


single

In [32]:
# traffic flow data reading
header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
          'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
          'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
          'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
          'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
          'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
          'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
          'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
          'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
          'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']

srs_file_name = '../data/PeMS/Series/d07_text_station_5min_2017_10_11.txt'
data_srs_no_null = pd.read_csv(srs_file_name, sep=',', names=header_srs, parse_dates=[0])
data_srs_no_null = data_srs_no_null[data_srs_no_null.columns[:-40]]
data_srs_no_null = data_srs_no_null.dropna(subset=['Avg Speed']).reset_index(drop=True)

In [33]:
data_srs_no_null.to_csv('../data/PeMS/Series/light/d07_text_station_5min_2017_10_11.csv', index=False)

In [34]:
data_srs_smoothed = data_srs_no_null[data_srs_no_null.columns[:16]].copy()
srs_for_smoothing_gb = data_srs_smoothed.groupby(by=['Station'])
j=0
for i, item in srs_for_smoothing_gb:
#     print(item['Avg Speed'].rolling(5).mean())
#     station = item.Station[0]
    data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
#     j += 1
#     if j==2:
#         break
data_srs_smoothed = data_srs_smoothed.dropna(subset=['Avg Speed'])
del srs_for_smoothing_gb

In [35]:
data_srs_smoothed.to_csv('../data/PeMS/Series/smoothed/d07_text_station_5min_2017_10_11.csv', index=False)

# Stations

In [1]:
def resave_stations(path):
    pth = Path(path + '/raw/')
    header_stations = ['ID', 'Freeway', 'Freeway dir', 'Country', 'City', 'State Postmile', 'Abs Postmile', 
          'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name',
          'User ID1', 'User ID2', 'User ID3', 'User ID4']
    for child in pth.iterdir():
        st_data_file_name = path + '/raw/' + child.name
        st_blacklist = np.lo
        stations_active = pd.read_csv(st_data_file_name, sep='\t')#, names=header_stations)
        stations_active = stations_active.astype({'Latitude':float, 'Longitude':float})
        stations_active = stations_active.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
        stations_active = stations_active.sort_values('Lanes', ascending=False).drop_duplicates(subset=['Latitude', 'Longitude'], keep='first').sort_index().reset_index(drop=True)
        stations_active = stations_active[~stations_active.ID.isin(st_blacklist)]
        stations_active.to_csv(path + '/light' + child.name[:-4] + '.csv', index=False)

In [4]:
resave_stations('../data/PeMS/Incidents/work_folder/Months/inc/stations/')

In [5]:
file_stations = '../data/PeMS/Stations/d07_text_meta_2017_09_20.txt'
header_stations = ['ID', 'Freeway', 'Freeway dir', 'Country', 'City', 'State Postmile', 'Abs Postmile', 
          'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name',
          'User ID1', 'User ID2', 'User ID3', 'User ID4']
stations_active = pd.read_csv(file_stations, sep='\t')
stations_active = stations_active.astype({'Latitude':float, 'Longitude':float})
stations_active = stations_active.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
# result: non-empty speed data series  only, stations which provide these data
stations_active = stations_active[stations_active['ID'].isin(data_srs_no_null['Station'].unique())].reset_index(drop=True)
stations_active = stations_active.sort_values('Lanes', ascending=False).drop_duplicates(subset=['Latitude', 'Longitude'], keep='first').sort_index().reset_index(drop=True)
stations_active = stations_active[~stations_active.ID.isin(st_blacklist)]

In [6]:
stations_active.to_csv('../data/PeMS/Stations/light/d07_text_meta_2017_09_20_active.csv', index=False)

# Do not use

In [38]:
# incidents week before data reading
incdnt_file_name = '../data/PeMS/Incidents/all_text_chp_incident_day_2017_10_04.txt'
data_inc_04_d07 = pd.read_csv(incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
data_inc_04_d07 = (data_inc_04_d07.dropna(subset=['District']))[data_inc_04_d07.columns[:-4]]
data_inc_04_d07 = data_inc_04_d07.astype(dtype={'District':int})
# result: incidents in district under analysis
data_inc_04_d07 = data_inc_04_d07[data_inc_04_d07['District']==7].reset_index(drop=True)

In [39]:
data_inc_04_d07.to_csv('../data/PeMS/Incidents/light/all_text_chp_incident_day_2017_10_04.csv', index=False)

In [40]:
# traffic week b4 flow data reading
srs_file_name = '../data/PeMS/Series/d07_text_station_5min_2017_10_04.txt'
data_srs_no_null_04 = pd.read_csv(srs_file_name, sep=',', names=header_srs, parse_dates=[0])
data_srs_no_null_04 = data_srs_no_null_04[data_srs_no_null_04.columns[:-40]]
# result: non-empty speed data series  only, stations which provide these data
data_srs_no_null_04 = data_srs_no_null_04.dropna(subset=['Avg Speed']).reset_index(drop=True)

In [41]:
data_srs_no_null_04.to_csv('../data/PeMS/Series/light/d07_text_station_5min_2017_10_04.csv', index=False)